# Process Raw Sensor Data Collected by Master_Script.py

In [1]:
%matplotlib notebook
import pandas as pd
from datetime import datetime
import csv

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import pdb

In [2]:
def reward_occupancy(observation_sequence, reward_type = 'IR_distance'):
        """
        Calculate reward based on occupancy i.e. the IRs data
        
        Args:
            observation_sequence (pd.DataFrame): observation
                1st column      : observation['timestamp']
                2nd-25th columns: 24 IRs
        Kwargs:
            reward_type (string): reward type
                1. 'IR_distance': based on IR distance from detected object to IR
                2. 'IR_state_ratio': the ratio of # of detected objects and all # 
                                     of IR sensors 
                3. 'IR_state_number': the number of detected objects
        
        Returns:
            observation_sequence (pd.DataFrame): add reward of each observation as a column 'reward_IR_distance'
                last column: observation_sequence['reward_IR_distance']
        """
        # Make here insistent with IR data
        #   1. 'IR_distance': based on IR distance from detected object to IR
        #   2. 'IR_state_ratio': the ratio of # of detected objects and all # 
        #                        of IR sensors 
        #   3. 'IR_state_number': the number of detected objects
        
        if reward_type == 'IR_distance':
            observation_sequence['reward_IR_distance'] = observation_sequence.iloc[:, 1:1+24].sum(axis=1)
        elif reward_type == 'IR_state_ratio':
            # TODO:
            pass
        elif reward_type == 'IR_state_number':
            # TODO:
            pass
        else:
            raise Exception('Please choose a proper reward type!')
        
        return observation_sequence

#### Load Raw Sensor Data Collected by Master_Script.py
1. load sensro data
2. Get start time
3. Convert relative time to timestamp
4. Convert timestamp to readable datatime
5. Drop last column which contain NON
6. Filter out data outside [1pm, 4pm]
7. Calculate reward

In [3]:
import glob
raw_sensor_data_files = glob.glob('raw_sensor_data_from_master_script/*.csv')
raw_sensor_data_files.sort()
raw_sensor_data = {}
for file in raw_sensor_data_files:
    print('Loading: {}'.format(file))
    # 1. load sensro data
    raw_sensor_data[file] = pd.read_csv(file, skiprows=4)
    
    # 2. Get start time
    with open(file, newline='') as f:
        reader = csv.reader(f)
        for i in range(4):
            tmp = next(reader)
            #print(tmp)
            if i == 2:
                line_with_time = tmp
    _, start_time = line_with_time[0].split(':')
    datetime_format = '%Y-%m-%d %H:%M:%S'
    start_time = float(start_time)
    start_time_datetime_format = datetime.fromtimestamp(start_time).strftime(datetime_format)
    #print('start_time_datetime_format = {}'.format(start_time_datetime_format))
    
    # 3. Convert relative time to timestamp
    raw_sensor_data[file]['timestamp'] = start_time + raw_sensor_data[file]['timestamp']
    
    # 4. Convert timestamp to readable datatime
    raw_sensor_data[file]['timestamp'] = raw_sensor_data[file]['timestamp'].apply(pd.Timestamp.fromtimestamp)
    
    # 5. drop last column which contain NON
    raw_sensor_data[file] = raw_sensor_data[file].drop([raw_sensor_data[file].columns[-1]], axis=1)
    
    # 6. filter out data outside [1pm, 4pm]
    date, time = start_time_datetime_format.split(' ')
    raw_sensor_data[file] = raw_sensor_data[file].loc[(raw_sensor_data[file]['timestamp'] >= date+' '+'13:00:00') & (raw_sensor_data[file]['timestamp'] <= date+' '+'16:00:00')]
    
    # 7. clculate reward
    raw_sensor_data[file] = reward_occupancy(raw_sensor_data[file], reward_type = 'IR_distance')
    
    # reset index to start from 0
    raw_sensor_data[file] = raw_sensor_data[file].reset_index()
    raw_sensor_data[file] = raw_sensor_data[file].drop([raw_sensor_data[file].columns[0]], axis=1)
    #print('Loading: done')

Loading: raw_sensor_data_from_master_script/sensor_data__September_14_2018_at_09-00-00.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_17_2018_at_10-20-36.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_18_2018_at_11-33-32.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_19_2018_at_09-33-44.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_20_2018_at_12-58-58.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_21_2018_at_12-59-00.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_24_2018_at_11-45-01.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_25_2018_at_11-17-27.csv
Loading: raw_sensor_data_from_master_script/sensor_data__September_26_2018_at_09-43-46.csv


In [4]:
#raw_sensor_data[list(raw_sensor_data.keys())[0]]

In [5]:
def plot_reward_signal(raw_sensor_data, filename):
    ymin=0
    ymax=3
    ystep=0.2
    
    plt.figure()
    plt.plot(raw_sensor_data['timestamp'], raw_sensor_data['reward_IR_distance'], 'r-', lw=1.5, alpha=0.6)

    plt.xlabel('time/second')
    plt.ylabel('reward')
    plt.ylim(ymin, ymax, ystep)
    plt.xticks(rotation=45, fontsize = 12)
    plt.title('Reward: {}'.format(filename))
    plt.grid(True, linestyle=':')
    plt.tight_layout()

In [6]:
for file in raw_sensor_data.keys():
    _, filename = file.split('/')
    plot_reward_signal(raw_sensor_data[file], filename)
    #print(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def separate_data_from_different_behavior(raw_sensor_data, date,
                                          experiment_schedule_table = 'experiment_schedule_table.csv'):
    """
    Args:
        raw_sensor_data (pd.DataFrame): sensor data collect in one day
        experiment_schedule_table (string): file name of experiment schedule table which contains schedule of 
            each day's experiments in where:
                0: Prescribed_Behavior
                1: Parameterized_Learning_Agent
                2: Single_Agent_Raw_Act
                3: Agent_Community_Raw_Act
    Return:
        separated_data (list):
    """
    # read experiment schedule file and retrive the schedule of date
    experiment_schedule = pd.read_csv(experiment_schedule_table)
    phaseschedule = experiment_schedule.loc[experiment_schedule['Date'] == date]
    # read schedule code in each interaction phase
    phase_1pm_2pm_code = phaseschedule['1pm-2pm']
    phase_2pm_2_30pm_code = phaseschedule['2pm-2:30pm']
    phase_2_30pm_3pm_code = phaseschedule['2:30pm-3pm']
    phase_3pm_4pm_code = phaseschedule['3pm-4pm']
    # separate data
    pdb.set_trace()
    start_time = float(start_time)
    datetime_format = '%Y-%m-%d %H:%M:%S'
    start_time_datetime_format = datetime.fromtimestamp(raw_sensor_data['timestamp'][0]).strftime(datetime_format)
    date, time = raw_sensor_data['timestamp'][0].split(' ')
    
    phase_1pm_2pm_data = raw_sensor_data.loc[(raw_sensor_data['timestamp'] >= date+' '+'13:00:00') & (raw_sensor_data['timestamp'] <= date+' '+'14:00:00')]
    phase_2pm_2_30pm_data = raw_sensor_data.loc[(raw_sensor_data['timestamp'] >= date+' '+'14:00:00') & (raw_sensor_data['timestamp'] <= date+' '+'14:30:00')]
    phase_2_30pm_3pm_data = raw_sensor_data.loc[(raw_sensor_data['timestamp'] >= date+' '+'14:30:00') & (raw_sensor_data['timestamp'] <= date+' '+'15:00:00')]
    phase_3pm_4pm_data = raw_sensor_data.loc[(raw_sensor_data['timestamp'] >= date+' '+'15:00:00') & (raw_sensor_data['timestamp'] <= date+' '+'16:00:00')]
    pdb.set_trace()
    return 
    
def plot_reward_signal(raw_sensor_data, filename):
    """
    Args:
        raw_sensor_data (pd.DataFrame): the data
        filename (string): the name of the date file
    """
    ymin=0
    ymax=3
    ystep=0.2
    
    # separate data from different behavior
    # find the date of raw_sensor_data
    _, date = filename.split('_2018')[0].split('sensor_data__')
    pdb.set_trace()
    separate_data_from_different_behavior(raw_sensor_data, date,
                                          experiment_schedule_table = 'experiment_schedule_table.csv')
    
    # TODO: if 2pm-2:30pn and 2:30pm-3pm have the same schedule code, combine them.
    # if 1pm-2pm and 2pm-2:30 have the same schedule code, combine them.
    
    phase_1pm_2pm_time = raw_sensor_data['timestamp'][:20000]
    phase_1pm_2pm_reward = raw_sensor_data['reward_IR_distance'][:20000]
    
    phase_2pm_3pm_time = raw_sensor_data['timestamp'][20000:40000]
    phase_2pm_3pm_reward = raw_sensor_data['reward_IR_distance'][20000:40000]
    
    phase_3pm_4pm_time = raw_sensor_data['timestamp'][40000:]
    phase_3pm_4pm_reward = raw_sensor_data['reward_IR_distance'][40000:]
    
    # plot data
    plt.figure()
    
    plt.plot(phase_1pm_2pm_time, phase_1pm_2pm_reward, 'r-', lw=1.5, alpha=0.6)
    plt.plot(phase_2pm_3pm_time, phase_2pm_3pm_reward, 'b-', lw=1.5, alpha=0.6)
    plt.plot(phase_3pm_4pm_time, phase_3pm_4pm_reward, 'g-', lw=1.5, alpha=0.6)
    
    plt.legend()
    
    plt.xlabel('time/second')
    plt.ylabel('reward')
    plt.ylim(ymin, ymax, ystep)
    plt.xticks(rotation=45, fontsize = 12)
    plt.title('Reward: {}'.format(filename))
    plt.grid(True, linestyle=':')
    plt.tight_layout()

file = list(raw_sensor_data.keys())[7]
_, filename = file.split('/')

plot_reward_signal(raw_sensor_data[file], filename)

> <ipython-input-8-3b3e954adb76>(47)plot_reward_signal()
-> separate_data_from_different_behavior(raw_sensor_data, date,
(Pdb) c
> <ipython-input-8-3b3e954adb76>(25)separate_data_from_different_behavior()
-> date, time = raw_sensor_data['timestamp'][0].split(' ')
(Pdb) raw_sensor_data
                       timestamp     IR1-1     IR1-2     IR2-1     IR2-2  \
0     2018-09-25 13:00:00.008425  0.000978  0.000978  0.019550  0.464321   
1     2018-09-25 13:00:00.260280  0.000978  0.010753  0.036168  0.472141   
2     2018-09-25 13:00:00.379212  0.004888  0.002933  0.024438  0.478006   
3     2018-09-25 13:00:00.480169  0.004888  0.002933  0.024438  0.478006   
4     2018-09-25 13:00:00.581117  0.004888  0.002933  0.024438  0.478006   
5     2018-09-25 13:00:00.682039  0.004888  0.002933  0.024438  0.478006   
6     2018-09-25 13:00:00.782982  0.004888  0.002933  0.024438  0.478006   
7     2018-09-25 13:00:00.883924  0.004888  0.002933  0.024438  0.478006   
8     2018-09-25 13:00:00.9848

(Pdb) raw_sensor_data['timestamp'][0]
Timestamp('2018-09-25 13:00:00.008425')
(Pdb) type(raw_sensor_data['timestamp'][0])
<class 'pandas._libs.tslib.Timestamp'>


In [ ]:
experiment_schedule_table = 'experiment_schedule_table.csv'
experiment_schedule = pd.read_csv(experiment_schedule_table)
experiment_schedule

In [ ]:
_, date = file.split('_2018')[0].split('sensor_data__')
phaseschedule = experiment_schedule.loc[experiment_schedule['Date'] == date]
phaseschedule

In [ ]:
phase_1pm_2pm = phaseschedule['1pm-2pm']
phase_2pm_2_30pm = phaseschedule['2pm-2:30pm']
phase_2_30pm_3pm = phaseschedule['2:30pm-3pm']
phase_3pm_4pm = phaseschedule['3pm-4pm']

In [ ]:
print(phase_1pm_2pm == 0)

In [ ]:
experiment_schedule['Date']